In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor

df = pd.read_csv(r'C:\Users\Kerry\masaustu_data_kaggle.csv')
df.head()

,Marka,Fiyat,İşlemci Tipi,SSD Kapasitesi,Ram (Sistem Belleği),Ekran Kartı,Kapasite,İşletim Sistemi,Ekran Kartı Bellek Tipi,Ekran Kartı Tipi,...,Ekran Kartı Hafızası,Temel İşlemci Hızı (GHz),Bağlantılar,Cihaz Ağırlığı,Ekran Boyutu,İşlemci Frekansı,Ekran Yenileme Hızı,Panel Tipi,Menşei,Arttırılabilir Azami Bellek
0,XASER,13716.90,Intel Core i5,512 GB,16 GB,AMD Radeon RX 550,512 GB,Free Dos (İşletim Sistemi Yok),GDDR5,Harici,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DMC,13678.00,Intel Core i5,512 GB,16 GB,Nvidia Geforce GT 740,Yok,Windows,DDR3,Harici,...,4 GB ve altı,3.2,HDMI,Belirtilmemiş,24 inç,3.00 GHz üstü,75 Hz,NaN,NaN,NaN
2,XASER,12857.74,Intel Core i5,512 GB,16 GB,AMD Radeon RX 580,512 GB,Free Dos (İşletim Sistemi Yok),GDDR5,Harici,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DMC,20799.00,Intel Core i5,256 GB,32 GB,AMD Radeon RX 580,256 GB,Windows,GDDR5,Harici,...,8 GB,2.9,HDMI,Belirtilmemiş,"24"" / 61 Ekran",NaN,165 Hz,VA,TR,NaN
4,XASER,12834.62,Intel Core i5,512 GB,16 GB,AMD Radeon RX 550,512 GB,Free Dos (İşletim Sistemi Yok),GDDR5,Harici,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Random Forest

In [10]:
targets = "Fiyat"
features = [df.columns[12], df.columns[19]]

# Dropping rows with missing target values
df = df.dropna(subset=[targets])

# Filling missing values in selected features
df[features] = df[features].fillna(df[features].mode().iloc[0])

# Encoding categorical variables
encoder = LabelEncoder()
for col in features:
    df[col] = encoder.fit_transform(df[col])

# Scaling numerical features
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

# Removing outliers in the target column (Fiyat) using IQR
Q1 = df[targets].quantile(0.25)
Q3 = df[targets].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df = df[(df[targets] >= lower_bound) & (df[targets] <= upper_bound)]

# Splitting data into training and testing sets
X = df[features].iloc[:, :]
y = df[targets]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Making predictions
y_pred = model.predict(X_test)

# Evaluating the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Printing results
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

Mean Absolute Error (MAE): 5801.288696274706
Root Mean Squared Error (RMSE): 7498.431476239387
